In [5]:
import pandas as pd
import requests
import os
import json
import time
from typing import List, Dict, Any, Optional
from dataclasses import dataclass

# BlockVision API key from our memory
BLOCKVISION_API_KEY = "2uulalvqIxowwmCkEMGozKfUmrW"

/Users/cyber/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [16]:
@dataclass
class NFTData:
    """Class to store NFT metadata"""
    object_id: str
    nft_name: str
    image_url: str
    project_url: str = ""
    symbol: str = ""
    creator: str = ""
    description: str = ""


def get_all_nft_object_ids(collection_type: str, limit: int = 20) -> List[str]:
    """Get all NFT object IDs for a given collection type using BlockVision API"""
    url = "https://api.blockvision.org/v1/sui/mainnet/nft/collection/nfts"
    
    params = {
        "collection": collection_type,
        "page": 1,
        "limit": limit
    }
    
    headers = {
        "accept": "application/json",
        "X-API-KEY": BLOCKVISION_API_KEY
    }
    
    try:
        print(f"Fetching NFT object IDs for collection {collection_type}...")
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        
        data = response.json()
        if "data" in data and "nfts" in data["data"]:
            object_ids = [nft["objectId"] for nft in data["data"]["nfts"]]
            print(f"Found {len(object_ids)} NFT object IDs")
            return object_ids
        else:
            print(f"Unexpected response format from BlockVision API for collection {collection_type}")
            return []
    except Exception as e:
        print(f"Error fetching NFT object IDs for {collection_type}: {str(e)}")
        return []


def get_nft_data_by_object_id(object_id: str) -> Optional[NFTData]:
    """Get NFT data for a single object ID using BlockVision API"""
    url = f"https://api.blockvision.org/v1/sui/mainnet/nft/object/{object_id}"
    
    headers = {
        "accept": "application/json",
        "X-API-KEY": BLOCKVISION_API_KEY
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        data = response.json()
        if "data" in data and data["data"]:
            nft_data = data["data"]
            return NFTData(
                object_id=object_id,
                nft_name=nft_data.get("name", ""),
                image_url=nft_data.get("imageUrl", ""),
                project_url=nft_data.get("projectUrl", ""),
                symbol=nft_data.get("symbol", ""),
                creator=nft_data.get("creator", ""),
                description=nft_data.get("description", "")
            )
        else:
            print(f"No data found for NFT {object_id}")
            return None
    except Exception as e:
        print(f"Error fetching NFT data for {object_id}: {str(e)}")
        return None


def get_all_nft_datas_by_multiGetObjectsAPI(object_ids: List[str], batch_size: int = 5) -> List[NFTData]:
    """Get NFT data for multiple object IDs using batch requests"""
    all_nft_data = []
    
    # Process in batches to avoid rate limiting
    for i in range(0, len(object_ids), batch_size):
        batch = object_ids[i:i+batch_size]
        print(f"Processing batch {i//batch_size + 1}/{(len(object_ids) + batch_size - 1)//batch_size}")
        
        for object_id in batch:
            nft_data = get_nft_data_by_object_id(object_id)
            if nft_data:
                all_nft_data.append(nft_data)
            
            # Add a small delay to avoid rate limiting
            time.sleep(0.2)
        
        # Add a delay between batches
        if i + batch_size < len(object_ids):
            print("Waiting between batches...")
            time.sleep(1)
    
    return all_nft_data

In [17]:
# Load the CSV file with NFT types
df = pd.read_csv("1_all_nft_types.csv")
print(f"Loaded {len(df)} NFT collections from CSV")

# Display the first few rows
df.head()

Loaded 49 NFT collections from CSV


,nft_collection_name,nft_type,verified,tradeport_url,supply,floor,volume,cover_url,description,discord,twitter,website
0,Fuddies,0xac176715abe5bcdaae627c5048958bbe320a8474f524...,True,https://www.tradeport.xyz/sui/collection/0xac1...,10000,24000000000,3666054734229328,https://byzantion.mypinata.cloud/ipfs/QmaBCGFX...,"10,000 extra special owls.",NaN,https://twitter.com/FuddiesNFT,NaN
1,SuiFrens: Bullsharks,0x8894fa02fc6f36cbc485ae9145d05f247a78e220814f...,True,https://www.tradeport.xyz/sui/collection/0x889...,380667,1000,1855043140799099,https://byzantion.mypinata.cloud/ipfs/QmPKwiLy...,My SuiFren!,NaN,https://twitter.com/SuiFrens,https://sui.io/
2,SuiFrens: Capys,0xee496a0cc04d06a345982ba6697c90c619020de9e274...,True,https://www.tradeport.xyz/sui/collection/0xee4...,191171,500000000,1401307787268524,https://byzantion.mypinata.cloud/ipfs/QmSk9wP9...,My SuiFren!,NaN,https://twitter.com/SuiFrens,https://sui.io/
3,Prime Machin,0x034c162f6b594cb5a1805264dd01ca5d80ce3eca6522...,True,https://www.tradeport.xyz/sui/collection/0x034...,3333,367000000000,1078313626659289,https://img.sm.xyz/0xc94a37b7a848963fc6ae28bf8...,Prime Machin #1818 manufactured by the Triangl...,NaN,NaN,NaN
4,DeSuiLabs,0x4edaf43ada89b42ba4dee9fbf74a4dee3eb01f3cfd31...,True,https://www.tradeport.xyz/sui/collection/0x4ed...,2222,21700000000,744335261237764,https://shdw-drive.genesysgo.net/BpJJEY3UBobdC...,2222 degenerates running experiments on Sui,https://discord.com/invite/2Cfm9gpTCu,https://twitter.com/DeSuiLabs,https://winx.io/


In [19]:
# The five specific collections we're interested in
target_collections = ["AEON", "Prime Machin", "Rootlets", "Kumo", "DoubleUp Citizen"]
print(f"Looking for these specific collections: {', '.join(target_collections)}")

# Filter the DataFrame to include only our target collections
filtered_df = df[df['nft_collection_name'].isin(target_collections)]

# Check for duplicates
duplicate_collections = filtered_df['nft_collection_name'].value_counts()
print("\nCollection counts:")
for collection, count in duplicate_collections.items():
    print(f"- {collection}: {count} occurrences")

# Remove duplicates (keep first occurrence)
filtered_df = filtered_df.drop_duplicates(subset=['nft_collection_name'])

print(f"\nAfter removing duplicates, found {len(filtered_df)} unique collections:")
for idx, row in filtered_df.iterrows():
    print(f"- {row['nft_collection_name']} (Type: {row['nft_type']})")

filtered_df

Looking for these specific collections: AEON, Prime Machin, Rootlets, Kumo, DoubleUp Citizen

Collection counts:
- Rootlets: 2 occurrences
- Prime Machin: 1 occurrences
- Kumo: 1 occurrences
- AEON: 1 occurrences
- DoubleUp Citizen: 1 occurrences

After removing duplicates, found 5 unique collections:
- Prime Machin (Type: 0x034c162f6b594cb5a1805264dd01ca5d80ce3eca6522e6ee37fd9ebfb9d3ddca::factory::PrimeMachin)
- Rootlets (Type: 0x8f74a7d632191e29956df3843404f22d27bd84d92cca1b1abde621d033098769::rootlet::Rootlet)
- Kumo (Type: 0x57191e5e5c41166b90a4b7811ad3ec7963708aa537a8438c1761a5d33e2155fd::kumo::Kumo)
- AEON (Type: 0x141d8a2333f9369452fe075331924bb98d2abf0ee98de941db85aaf809c4ef54)
- DoubleUp Citizen (Type: 0x862810efecf0296db2e9df3e075a7af8034ba374e73ff1098e88cc4bb7c15437::doubleup_citizens::DoubleUpCitizen)


,nft_collection_name,nft_type,verified,tradeport_url,supply,floor,volume,cover_url,description,discord,twitter,website
3,Prime Machin,0x034c162f6b594cb5a1805264dd01ca5d80ce3eca6522...,True,https://www.tradeport.xyz/sui/collection/0x034...,3333,367000000000,1078313626659289,https://img.sm.xyz/0xc94a37b7a848963fc6ae28bf8...,Prime Machin #1818 manufactured by the Triangl...,NaN,NaN,NaN
7,Rootlets,0x8f74a7d632191e29956df3843404f22d27bd84d92cca...,True,https://www.tradeport.xyz/sui/collection/0x8f7...,3333,168000000000,503319122816221,ipfs://QmWFurw6xktFkSvKRLWwoNHtPcUBGJDpimECYiz...,Rootlets,NaN,NaN,NaN
9,Kumo,0x57191e5e5c41166b90a4b7811ad3ec7963708aa537a8...,True,https://www.tradeport.xyz/sui/collection/0x571...,2222,74000000000,385654143955323,ipfs://QmQRE6ymYcd7Z3kMRPLDGRSyW4zKDMGRbom5EaF...,Minted at the Kumo NFT launch. The cat has ent...,NaN,NaN,NaN
13,AEON,0x141d8a2333f9369452fe075331924bb98d2abf0ee98d...,True,https://www.tradeport.xyz/sui/collection/0x141...,993,163990000000,274228292831420,"data:image/webp;base64,UklGRroIAABXRUJQVlA4TK0...","A collection of 1000 unique, pixelated charact...",NaN,NaN,NaN
19,DoubleUp Citizen,0x862810efecf0296db2e9df3e075a7af8034ba374e73f...,True,https://www.tradeport.xyz/sui/collection/0x862...,2878,84000000000,160397182837707,ipfs://QmPmNdHofDDpCC2qAKUReVussmKergGa8GyNakr...,A citizen on the DoubleUp Island!,NaN,NaN,NaN
